In [1]:
import numpy as np

In [157]:
with open('./input.txt') as f:
    items = [r.strip().split(' -> ') for r in f]

In [173]:
class pt:
    
    def __init__(self, coord_str):
        x_str, y_str = coord_str.split(',')
        self.x = int(x_str)
        self.y = int(y_str)
    
    def is_orthogonal_to(self, other_pt):
        return self.x == other_pt.x or self.y == other_pt.y
    
    def __str__(self):
        return f"({self.x},{self.y})"
    
    
class VentLine:
    
    def __init__(self, file_line):
        from_str, to_str = file_line.strip().split(' -> ')
        self.from_pt = pt(from_str)
        self.to_pt = pt(to_str)
    
    def __str__(self):
        return f"{str(self.from_pt)} -> {str(self.to_pt)}"
    
    def is_orthogonal(self) -> bool:
        return self.from_pt.is_orthogonal_to(self.to_pt)
    
    def is_horizontal(self) -> bool:
        return self.from_pt.y == self.to_pt.y
    
    def min_coords(self):
        return (min(self.from_pt.x, self.to_pt.x), min(self.from_pt.y, self.to_pt.y))
    
    def max_coords(self):
        return (max(self.from_pt.x, self.to_pt.x), max(self.from_pt.y, self.to_pt.y))
    
    def draw_line_on_seabed(self, seabed_array):
        if not self.is_orthogonal():
            # could adapt this part2 solution to do part1 as well, just change the arange step
            dx = np.sign(self.to_pt.x - self.from_pt.x)
            dy = np.sign(self.to_pt.y - self.from_pt.y)
            xcoords = np.arange(self.from_pt.x, self.to_pt.x + dx, dx)
            ycoords = np.arange(self.from_pt.y, self.to_pt.y + dy, dy)
            # NB we can't return the index-based slice and update it in the caller as that would 
            # be a copy not a view so the array would not be affected
            seabed_array[ycoords, xcoords] += 1
        
        elif self.is_horizontal():
            # we could return the slice and do the increment it in caller as it's a view
            seabed_array[self.from_pt.y, 
                        self.min_coords()[0]: self.max_coords()[0]+1] += 1

        else:
            # we could return this slice and do the increment in the caller as it's a view
            seabed_array[self.min_coords()[1] : self.max_coords()[1]+1, 
                        self.from_pt.x] += 1
        

In [174]:
vent_lines = [VentLine(line) for line in open('./input.txt').readlines()]
len(vent_lines)

500

In [175]:
orthog_lines = [v for v in vent_lines if v.is_orthogonal()]
len(orthog_lines)

347

In [168]:
diag_lines = [v for v in vent_lines if not v.is_orthogonal()]
len(diag_lines)

153

In [176]:
seabed_size_x = max([v.max_coords()[0] for v in vent_lines])
seabed_size_y = max([v.max_coords()[1] for v in vent_lines])

In [177]:
seabed_size_x, seabed_size_y

(989, 989)

In [178]:
board = np.zeros(shape=(seabed_size_y, seabed_size_x), dtype=np.int32)
for ventline in orthog_lines:
    ventline.draw_line_on_seabed(board)
np.sum(board>1)

7644

In [188]:
board = np.zeros(shape=(1000, 1000), dtype=np.int32)
for ventline in vent_lines:
    ventline.draw_line_on_seabed(board)
np.sum(board>1)

18627

### Time HG part 2 solution

In [189]:
%%timeit
board = np.zeros(shape=(1000, 1000), dtype=np.int32)
for ventline in vent_lines:
    ventline.draw_line_on_seabed(board)
np.sum(board>1)


5.5 ms ± 79.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



## JY: totally different solution based on Rasterio

In [182]:
from shapely import wkt
from rasterio.features import rasterize
from rasterio.enums import MergeAlg

vents = [wkt.loads(
    f"LINESTRING ({line.replace(',', ' ').replace(' -> ', ', ')})"
) for line in data]
orthogonal_vents = [vent for vent in vents if len(set(vent.bounds)) < 4]
np.sum(rasterize(orthogonal_vents, out_shape=(1000,1000), merge_alg=MergeAlg("ADD")) >=2)

7644

In [190]:
np.sum(rasterize(vents, out_shape=(1000,1000), merge_alg=MergeAlg("ADD")) >=2)

18627

### Time JY rio solution

Only 50% slower, well worth it for the ease!

In [191]:
%%timeit
np.sum(rasterize(vents, out_shape=(1000,1000), merge_alg=MergeAlg("ADD")) >=2)

8.19 ms ± 95.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Slick solution from reddit

Looks nice but way slower

In [184]:
ls = np.fromregex(open('./input.txt'), '\d+', [('',int)]*4)

In [187]:
%%timeit
grid = np.zeros((2, 1000, 1000))
for (x, y, X, Y) in ls:
    dx, dy = np.sign([X-x, Y-y])                 
    while (x,y) != (X+dx, Y+dy):
        grid[dx * dy, x, y] += 1
        x+=dx; y+=dy

#print((grid[0]>1).sum(), (grid.sum(0)>1).sum())
(grid.sum(0)>1).sum()

174 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
